# Bibliotecas

In [1]:
import ranking

import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import tensorflow_hub as hub
import tensorflow_text

import time

In [4]:
#import os
#os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "UNCOMPRESSED"

In [8]:
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

# Constantes

In [9]:
SEED = 42
VAR_USEM = (0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9)

# Dataset

In [22]:
#arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
#arquivos = ['ale_1_1']
#arquivos = ['ale_5_1']
arquivos = ['hn_1_1']
#arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_teste.append(df_teste)

## Funções

In [23]:
def calcular_y_pred(limite, similaridade):

    y_pred = [1 if num >= limite else 0 for num in similaridade]

    return y_pred


def salvar_df_resultado(report, modelo, nome_dataset, tempo_exec):
    
    df_resultado = pd.DataFrame(report).transpose()
    df_resultado['modelo'] = modelo
    df_resultado['dataset'] = nome_dataset
    df_resultado['tempo'] = tempo_exec

    df_resultado.to_csv(f'Dados/Resultados/{modelo}/{nome_dataset}_resultado.csv', index = True)
    
    return df_resultado


def montar_df_resultado(y_teste, y_pred, df_teste, nome):
    df_y = pd.DataFrame(
                        list(zip(
                                 y_teste, y_pred,
                                 df_teste["categoria"].to_list(),
                                 df_teste["titulo_1"].to_list(),
                                 df_teste["titulo_2"].to_list()
                                )
                       ), columns = ['match', 'pred', 'categoria', 'titulo_1', 'titulo_2'])

    return df_y


def salvar_sim_dis(y_teste, similaridade, df_teste, nome, dis_sim = "Distância"):

    df_y = montar_df_resultado(y_teste, similaridade, df_teste, nome)
    df_y.to_csv(f'Dados/Resultados/USEM/{dis_sim}/{nome}_similaridade.csv', index = False)


def salvar_y_pred(y_teste, y_pred, df_teste, nome, limite, dis_sim = "Distância"):

    df_y = montar_df_resultado(y_teste, y_pred, df_teste, nome)
    df_y.to_csv(f'Dados/Resultados/USEM/{dis_sim}/USEM_{limite}/Resultado/{nome}_y.csv', index = False)


def salvar_relatorio(y_teste, y_pred, nome, tempo, limite, dis_sim = "Distância"):

    relatorio = classification_report(y_teste, y_pred, output_dict = True)
    df_relatorio = pd.DataFrame(relatorio).transpose()
    df_relatorio['modelo'] = nome
    df_relatorio['tempo'] = tempo

    df_relatorio.to_csv(f'Dados/Resultados/USEM/{dis_sim}/USEM_{limite}/Relatório/{nome}_relatório.csv', index = True)

In [24]:
def pipeline_usem(titulo_1, titulo_2):
    
    # model initialization
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
    
    # Calculando embeddings
    embedding_1 = embed(titulo_1)
    embedding_2 = embed(titulo_2)

    similaridade = []
    distancia = []
    for i in range(len(embedding_1)):
        # Calculando a matriz de similaridade. Quanto maior o score maior a similaridade.
        similaridade.append(np.inner(embedding_1[i], embedding_2[i]))
        distancia.append(ranking.calcular_dis_cos(embedding_1[i], embedding_2[i]))

    return similaridade, distancia

## Universal Sentence Encoder Multilingual

In [25]:
flag_usem = True
if flag_usem == True:

    #tam = 10
    for nome, df_teste in zip(arquivos, lista_df_teste):
        
        X_teste_1 = df_teste["titulo_1"]
        X_teste_2 = df_teste["titulo_2"]
        y_teste = df_teste["match"].to_list()

        inicio_tempo = time.time()
        #y_teste = y_teste[:tam]
        #similaridade, distancia = pipeline_usem(X_teste_1[:tam], X_teste_2[:tam])
        similaridade, distancia = pipeline_usem(X_teste_1, X_teste_2)
        final_tempo = time.time()
        tempo = final_tempo - inicio_tempo

        salvar_sim_dis(y_teste, similaridade, df_teste, nome, "Similaridade")
        salvar_sim_dis(y_teste, distancia, df_teste, nome)

        for limite in VAR_USEM:

            y_pred_sim = calcular_y_pred(limite, similaridade)

            salvar_y_pred( y_teste, y_pred_sim, df_teste, nome, ranking.remove_pontuacao(str(limite)), "Similaridade" )
            salvar_relatorio( y_teste, y_pred_sim, nome, tempo, ranking.remove_pontuacao(str(limite)), "Similaridade" )

            y_pred_dis = calcular_y_pred(limite, distancia)

            salvar_y_pred( y_teste, y_pred_dis, df_teste, nome, ranking.remove_pontuacao(str(limite)) )
            salvar_relatorio( y_teste, y_pred_dis, nome, tempo, ranking.remove_pontuacao(str(limite)) )